✅ الهدف:
تطبيق Flutter فيه:

🎤 زر تسجيل صوتي (عربي)

📝 التعرف على النص

🔤 استخراج الكلمات

📼 تشغيل فيديوهات لغة الإشارة (من جهاز المستخدم)

بدون سيرفر - بدون اتصال إنترنت

🧱 1. إعداد هيكل الملفات
في مشروع Flutter، أنشئ المجلدات التالية:


#videos جميع الفيديوهات ضعها في ملف

In [ ]:
my_flutter_app/
├── assets/
│   └── videos/       ✅ ضَع هنا جميع ملفات الفيديو mp4
│       ├── Hello.mp4
│       ├── Welcome.mp4
│       └── ...
├── lib/
│   └── main.dart     ✅ سنضيف الكود هنا
├── pubspec.yaml


🛠️ 2. إعداد pubspec.yaml


افتح الملف وأضف التالي:

In [ ]:
dependencies:
  flutter:
    sdk: flutter
  speech_to_text: ^5.10.0
  video_player: ^2.5.1

flutter:
  assets:
    - assets/videos/


ثم شغّل:

In [ ]:
flutter pub get


🎯 3. كود main.dart


انسخ هذا الكود إلى lib/main.dart:

In [ ]:
import 'package:flutter/material.dart';
import 'package:speech_to_text/speech_to_text.dart' as stt;
import 'package:video_player/video_player.dart';

void main() {
  runApp(SignLangApp());
}

class SignLangApp extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Sign Language App',
      theme: ThemeData.dark(),
      home: SignHomePage(),
    );
  }
}

class SignHomePage extends StatefulWidget {
  @override
  _SignHomePageState createState() => _SignHomePageState();
}

class _SignHomePageState extends State<SignHomePage> {
  final stt.SpeechToText _speech = stt.SpeechToText();
  bool _isListening = false;
  String _recognizedText = '';
  List<String> videoWords = [];
  int currentIndex = 0;
  VideoPlayerController? _videoController;

  void _listenAndConvert() async {
    bool available = await _speech.initialize();
    if (available) {
      setState(() => _isListening = true);
      _speech.listen(
        localeId: 'ar_SA',
        onResult: (val) async {
          if (val.finalResult) {
            setState(() {
              _recognizedText = val.recognizedWords;
              _isListening = false;
            });

            final words = _extractWords(_recognizedText);
            setState(() {
              videoWords = words;
              currentIndex = 0;
            });

            if (videoWords.isNotEmpty) {
              await _playVideo(videoWords[0]);
            }
          }
        },
      );
    }
  }

  List<String> _extractWords(String text) {
    final cleanText = text.replaceAll(RegExp(r'[^\w\s]'), '');
    return cleanText.split(' ').where((w) => w.trim().isNotEmpty).toList();
  }

  Future<void> _playVideo(String word) async {
    _videoController?.dispose();
    final path = 'assets/videos/${word[0].toUpperCase()}${word.substring(1)}.mp4';

    _videoController = VideoPlayerController.asset(path);

    try {
      await _videoController!.initialize();
      setState(() {});
      _videoController!.play();

      _videoController!.addListener(() async {
        if (_videoController!.value.position >= _videoController!.value.duration &&
            currentIndex < videoWords.length - 1) {
          currentIndex++;
          await _playVideo(videoWords[currentIndex]);
        }
      });
    } catch (e) {
      print('❌ Video not found: $path');
      if (currentIndex < videoWords.length - 1) {
        currentIndex++;
        await _playVideo(videoWords[currentIndex]);
      }
    }
  }

  @override
  void dispose() {
    _videoController?.dispose();
    super.dispose();
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('🎤 Text to Sign Language')),
      body: Padding(
        padding: const EdgeInsets.all(16.0),
        child: Column(
          children: [
            Text(
              _recognizedText.isEmpty ? 'اضغط المايك وابدأ التحدث' : _recognizedText,
              textAlign: TextAlign.center,
              style: TextStyle(fontSize: 20),
            ),
            SizedBox(height: 20),
            ElevatedButton.icon(
              icon: Icon(Icons.mic),
              label: Text(_isListening ? 'يسجّل...' : 'اضغط للتسجيل'),
              onPressed: _isListening ? null : _listenAndConvert,
              style: ElevatedButton.styleFrom(
                backgroundColor: Colors.amber,
                foregroundColor: Colors.black,
              ),
            ),
            SizedBox(height: 20),
            _videoController != null && _videoController!.value.isInitialized
                ? AspectRatio(
                    aspectRatio: _videoController!.value.aspectRatio,
                    child: VideoPlayer(_videoController!),
                  )
                : Container(
                    height: 200,
                    color: Colors.black12,
                    child: Center(child: Text('لا يوجد فيديو')),
                  ),
          ],
        ),
      ),
    );
  }
}


✅ 4. تشغيل المشروع
افتح المحاكي أو وصل جوالك

ثم شغل المشروع:

In [ ]:
flutter run
